<a href="https://colab.research.google.com/github/hollywgnr/cs490Ahw3/blob/main/HW3BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 3 BERT, cs490A, Holly Wagner

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 3.1 Tokenization
Below I pass a sentence to BERT’s tokenizer which produces a vocabulary with some continuation tokens. Continuation tokens start with '##' and occur when the original word has been split into smaller subwords and characters. For example, the word "chimpanzee" contains subwords like 'chi', 'mp', 'an' and 'zee' which may occur in other words or have meaning by themselves.

In [ ]:
txt = "Hey, look its a Chimpanzee!"
marked_text = "[CLS] " + txt + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'hey', ',', 'look', 'its', 'a', 'chi', '##mp', '##an', '##zee', '!', '[SEP]']


# 3.2 BERT features for text classification
note: I followed this tutorial
http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/hollywgnr/cs490Ahw3/main/final.csv', delimiter=',')

In [ ]:
df.head()

,label,text
0,MV,best movie ive seen in years.
1,OTH,You can't be over 17 if you are worried about ...
2,MV,Parents having a hard time letting you watch R...
3,OTH,"Sure, my intent is not to let anyone down."
4,OTH,I am just saying your apprehensions are irrati...


In [ ]:
df.columns

Index(['label', 'text'], dtype='object')

In [ ]:
df['text']

0                          best movie ive seen in years.
1      You can't be over 17 if you are worried about ...
2      Parents having a hard time letting you watch R...
3             Sure, my intent is not to let anyone down.
4      I am just saying your apprehensions are irrati...
                             ...                        
245                    So rule-based matching for those?
246    Some of us have less than ideal learning condi...
247    Email your professors, they should be accommod...
248    I'll look into that, sounds like exactly what ...
249    When the kid first saw it, it was breathing in...
Name: text, Length: 250, dtype: object

In [ ]:
df['label']

0       MV
1      OTH
2       MV
3      OTH
4      OTH
      ... 
245    CLG
246    CLG
247    CLG
248    OTH
249    OTH
Name: label, Length: 250, dtype: object

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids)

In [ ]:
 # Slice the output for the first position for all the sequences, take all hidden unit outputs
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features.shape

(250, 768)

In [ ]:
labels = df['label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.6349206349206349